<a href="https://colab.research.google.com/github/is0280fp/google_colab/blob/main/GaitRec_get_csv_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [80]:
import csv
import tensorflow as tf
import functools
import numpy as np

# from IPython.core.debugger import Pdb; Pdb().set_trace()

# F_AP_PRO_left = open('/content/drive/MyDrive/GaitRect/GaitRect/GRF_F_AP_PRO_left.csv', encoding='utf-8')
# F_AP_PRO_left = csv.DictReader(F_AP_PRO_left)

# F_AP_PRO_right = open('/content/drive/MyDrive/GaitRect/GaitRect/GRF_F_AP_PRO_right.csv', encoding='utf-8')
# F_AP_PRO_right = csv.DictReader(F_AP_PRO_right)

# F_ML_PRO_left = open('/content/drive/MyDrive/GaitRect/GaitRect/GRF_F_ML_PRO_left.csv', encoding='utf-8')
# F_ML_PRO_left = csv.DictReader(F_ML_PRO_left)

# F_ML_PRO_right = open('/content/drive/MyDrive/GaitRect/GaitRect/GRF_F_ML_PRO_right.csv', encoding='utf-8')
# F_ML_PRO_right = csv.DictReader(F_ML_PRO_right)

# F_V_PRO_left = open('/content/drive/MyDrive/GaitRect/GaitRect/GRF_F_V_PRO_left.csv', encoding='utf-8')
# F_V_PRO_left = csv.DictReader(F_V_PRO_left)

# F_V_PRO_right = open('/content/drive/MyDrive/GaitRect/GaitRect/GRF_F_V_PRO_right.csv', encoding='utf-8')
# F_V_PRO_right = csv.DictReader(F_V_PRO_right)

with open('/content/drive/MyDrive/GaitRect/GaitRect/GRF_F_AP_PRO_left.csv') as f:
    reader = csv.reader(f)
    GRF_F_AP_PRO_left = [row for row in reader]

with open('/content/drive/MyDrive/GaitRect/GaitRect/GRF_metadata.csv') as f:
    reader = csv.reader(f)
    metadata = [row for row in reader]

GRF_items = np.array(GRF_F_AP_PRO_left, dtype=str)[0,::]
meta_items = np.array(metadata, dtype=str)[0,::]

GRF = np.array(GRF_F_AP_PRO_left)[1:,::]
GRF = np.array(GRF, dtype=float)

In [81]:
# meta = np.array(meta, dytpe=float) str型や日付など変換できない列があるので、逐次floatに変換忘れない
# metadataの前処理, 3(class_label_detail)と16(year.date)列目削除して
metadata_array = np.array(metadata)
meta = np.array(metadata)[1:,::]
meta = np.delete(meta, [3,16], axis=1)
# 2列目を5クラスに対応した数字に変換(HC=0, H=1, K=2, A=3, C=4)
meta = np.where(meta=='HC', 0, meta)
meta = np.where(meta=='H', 1, meta)
meta = np.where(meta=='K', 2, meta)
meta = np.where(meta=='A', 3, meta)
meta = np.where(meta=='C', 4, meta)
# meta.shape (8971, 18)
# NaNがあるとint変換できないので、NaNをチェックする?
meta_list = []
for i in range(len(meta)):
  meta_list.append(meta[i,::].astype(np.float32))
meta = np.array(meta_list) # meta.shape (8971, 18)

In [ ]:
meta_items

array(['SUBJECT_ID', 'SESSION_ID', 'CLASS_LABEL', 'CLASS_LABEL_DETAILED',
       'SEX', 'AGE', 'HEIGHT', 'BODY_WEIGHT', 'BODY_MASS', 'SHOE_SIZE',
       'AFFECTED_SIDE', 'SHOD_CONDITION', 'ORTHOPEDIC_INSOLE', 'SPEED',
       'READMISSION', 'SESSION_TYPE', 'SESSION_DATE', 'TRAIN',
       'TRAIN_BALANCED', 'TEST'], dtype='<U20')

In [ ]:
GRF.shape

(75732, 104)

In [144]:
# GRF_and_metaはGRFにmetadataを3と16項目目を削除してものを足しただけ
# 75554~75566まで残す
GRF_and_meta[75553:75567,0] == GRF[75553:75567,0]
GRF_and_meta[65320:65732,0] == GRF[65320:65732,0]
valid_GRF = GRF[75554, ::].reshape(1, 104)
valid_GRF.shape
valid_GRF[0,1]

888812544.0

In [19]:
# SESSION_ID = 888812544 だけmetadataある、使えるデータ
for i in range(10):
  print(GRF_SESSION_ID[75553+i])

888812543.0
888812544.0
888812544.0
888812544.0
888812544.0
888812544.0
888812544.0
888812544.0
888812544.0
888812544.0


In [146]:
# GRF_and_meta[75472::,0] == GRF[75472::,0] これより、GRF[75473,::]から最後までがいらないデータ
# metadataにないデータをSESSION_IDにもとづいて削除する(888812536~888812635と999910365~999910375の連番、ただし888812544は除く)
# strategy: 888812544だけ取り出して保存しておくが、それ以外remove
temp_GRF = GRF
# lst = list(range(75473, 75732))
# temp_GRF = temp_GRF[:75474, ::]
# temp_GRF[75473,1] # 888812536.0
temp_GRF = temp_GRF[:75473, ::]
temp_GRF[75472,1] # 888812536.0
print(temp_GRF.shape)
# 保存しておいたのを足す
temp_GRF = np.append(temp_GRF, valid_GRF, axis=0)
print(temp_GRF.shape)

(75473, 104)
(75474, 104)


In [149]:
# 本番
GRF = GRF[:75473, ::]
GRF = np.append(GRF, valid_GRF, axis=0)

i = 0
meta_SESSION_ID = meta[::,1]
GRF_SESSION_ID = GRF[::,1]
meta_SUBJECT_ID = meta[::,0]
GRF_SUBJECT_ID = GRF[::,0]
# GRF_and_meta = np.empty((len(GRF), 120))
GRF_and_meta = np.ones((len(GRF), 120))
for i in range(len(meta_SUBJECT_ID)):
  # from IPython.core.debugger import Pdb; Pdb().set_trace()
  # index = np.array(np.where(GRF_SUBJECT_ID == meta_SUBJECT_ID[i])[0]) # 問題
  index = np.array(np.where(GRF_SESSION_ID == meta_SESSION_ID[i])[0]) # 問題
  index = np.array(index)
  for k in index:
    GRF_and_meta[k, ::] = np.append(GRF[k, ::], meta[i, 2::])
  #k=0
# GRF_and_meta.shape (75732, 120)

In [150]:
np.allclose(GRF_and_meta[::,0:104], GRF) # データセットに加工する前のGRFとmetadataのセット完成!

True

In [151]:
GRF_and_meta.shape

(75474, 120)